In [1]:
#데이터 경로
#/home/cksgh8511/Electronic_Clustering/Main_code/data


In [2]:
import os

# 기본 경로 설정
BASE_DIR = os.getcwd()
DATA_DIR = BASE_DIR + "/data"

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from transformers import get_cosine_schedule_with_warmup
import math
import numpy as np
import pandas as pd
import numpy.typing as npt

import time
import datetime
import datautils
import tasks
from ts2vec import TS2Vec

from utils import init_dl_program, name_with_datetime, pkl_save, data_dropout

In [4]:
from ts2vec import TS2Vec
import datautils

# Load the ECG200 dataset from UCR archive
train_data, train_labels, test_data, test_labels = datautils.load_UCR('FordA')
#/home/cksgh8511/ts2vec/datasets/UCR/UCRArchive_2018/FordA/FordA_TRAIN.tsv
# (Both train_data and test_data have a shape of n_instances x n_timestamps x n_features)



In [5]:
print(train_data.shape)
print(train_labels.shape)

(3601, 500, 1)
(3601,)


In [6]:
train_data_flat = train_data.reshape(-1, 1)   # (3601*500, 1)
test_data_flat = test_data.reshape(-1, 1)

In [7]:
# Train a TS2Vec model
model = TS2Vec(
    input_dims=1,
    device=0,
    output_dims=320
)
loss_log = model.fit(
    train_data,
    n_epochs=10, 
    verbose=True
)

Epoch #0: loss=2.2356168219778274
Epoch #1: loss=0.7215545939074622
Epoch #2: loss=0.5151515041457282
Epoch #3: loss=0.4292923469013638
Epoch #4: loss=0.3664437876145045
Epoch #5: loss=0.394080245229933
Epoch #6: loss=0.2946092641684744
Epoch #7: loss=0.23115166392591263
Epoch #8: loss=0.19579250843988524
Epoch #9: loss=0.1910502701997757


In [8]:
import sys
sys.path.append('/home/cksgh8511/ts2vec')

from tasks import anomaly_detection
import importlib
importlib.reload(anomaly_detection)

<module 'tasks.anomaly_detection' from '/home/cksgh8511/ts2vec/tasks/anomaly_detection.py'>

In [9]:
# 1. Flatten (3601, 500, 1) → (3601*500, 1)
train_data_flat = train_data.reshape(-1, 1)
test_data_flat = test_data.reshape(-1, 1)

# 2. 라벨을 per-instance → per-timestamp 라벨로 확장
train_labels_flat = np.repeat(train_labels, train_data.shape[1])  # (3601 * 500,)
test_labels_flat = np.repeat(test_labels, test_data.shape[1])     # (1320 * 500,)

# 3. 타임스탬프 생성
train_ts = np.arange(len(train_data_flat))
test_ts = np.arange(len(test_data_flat))

# 4. dict로 포장
all_train_data = {'FordA': train_data_flat}
all_train_labels = {'FordA': train_labels_flat}
all_train_timestamps = {'FordA': train_ts}

all_test_data = {'FordA': test_data_flat}
all_test_labels = {'FordA': test_labels_flat}
all_test_timestamps = {'FordA': test_ts}

In [10]:
from tasks.anomaly_detection import eval_anomaly_detection
res_log, eval_res = eval_anomaly_detection(
    model,
    all_train_data,
    all_train_labels,
    all_train_timestamps,
    all_test_data,
    all_test_labels,
    all_test_timestamps,
    delay=7
)


print(res_log, eval_res)

[array([1, 0, 0, ..., 0, 0, 1])] {'f1': 0.6460657551368562, 'precision': 0.9926298852961022, 'recall': 0.4788732394366197, 'infer_time': 117.80465888977051}


In [11]:
# CSV 불러오기 (가정)
df = pd.read_csv('/home/cksgh8511/Electronic_Clustering/Main_code/data/case3b_th.csv', index_col=0)  # 행: 지역, 열: 시간

# 전처리
all_test_data = {}
all_test_labels = {}
all_test_timestamps = {}

for region_id in df.index:
    seq = df.loc[region_id].dropna().astype(np.float32).values
    if len(seq) < 50:  # 너무 짧은 시계열은 제외
        continue
    all_test_data[str(region_id)] = seq.reshape(-1, 1)  # (T, 1)
    all_test_labels[str(region_id)] = np.zeros(len(seq))  # 라벨 없음 → 모두 0
    all_test_timestamps[str(region_id)] = np.arange(len(seq))
all_test_data

{'1': array([[14.562711],
        [14.558345],
        [14.517982],
        ...,
        [15.010059],
        [14.930181],
        [14.808204]], dtype=float32),
 '2': array([[14.320858 ],
        [14.3222885],
        [14.2868395],
        ...,
        [14.782644 ],
        [14.708982 ],
        [14.595112 ]], dtype=float32),
 '3': array([[14.377786],
        [14.376777],
        [14.339314],
        ...,
        [14.833175],
        [14.756935],
        [14.639743]], dtype=float32),
 '4': array([[15.584745],
        [15.581099],
        [15.541931],
        ...,
        [16.031433],
        [15.952767],
        [15.832792]], dtype=float32),
 '5': array([[15.753959],
        [15.750663],
        [15.711945],
        ...,
        [16.20091 ],
        [16.122744],
        [16.003523]], dtype=float32),
 '6': array([[15.019774],
        [15.016743],
        [14.977709],
        ...,
        [15.469614],
        [15.391315],
        [15.271542]], dtype=float32),
 '7': array([[14.791502],
  

In [12]:
df

,0:00,0:05,0:10,0:15,0:20,0:25,0:30,0:35,0:40,0:45,...,23:10.6,23:15.6,23:20.6,23:25.6,23:30.6,23:35.6,23:40.6,23:45.6,23:50.6,23:55.6
id,,,,,,,,,,,,,,,,,,,,,
1,14.562711,14.558345,14.517981,14.441466,14.370639,14.318171,14.246281,14.198014,14.128908,14.060734,...,15.654141,15.556176,15.486700,15.399814,15.303138,15.206850,15.100097,15.010059,14.930181,14.808204
2,14.320858,14.322288,14.286839,14.216284,14.149974,14.103923,14.038991,13.996352,13.932256,13.868876,...,15.364745,15.276117,15.215312,15.138251,15.049357,14.960728,14.863937,14.782644,14.708983,14.595112
3,14.377785,14.376777,14.339314,14.266334,14.198175,14.149465,14.081664,14.036674,13.970526,13.905166,...,15.440957,15.348415,15.284033,15.202883,15.110780,15.019004,14.918101,14.833175,14.756935,14.639743
4,15.584746,15.581099,15.541931,15.467056,15.397366,15.346154,15.275771,15.228413,15.160551,15.093397,...,16.663356,16.566975,16.499121,16.413972,16.318993,16.224492,16.119863,16.031432,15.952768,15.832792
5,15.753959,15.750663,15.711945,15.637666,15.568403,15.517700,15.447906,15.400946,15.333551,15.266801,...,16.827870,16.732189,16.664995,16.580584,16.486273,16.392459,16.288670,16.200911,16.122743,16.003523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,14.860791,14.704949,14.575849,14.490209,14.444118,14.383212,14.301253,14.245287,14.212640,14.205210,...,15.413177,15.385680,15.278778,15.172455,15.154271,15.087798,14.999365,14.982300,15.010656,15.064318
1059,14.855091,14.699241,14.570043,14.484070,14.437801,14.376534,14.294208,14.238024,14.205034,14.197302,...,15.406096,15.377951,15.270728,15.163984,15.145187,15.078242,14.989196,14.971466,14.999244,15.052237
1060,23.831147,23.681672,23.565127,23.514018,23.491069,23.470925,23.432325,23.408411,23.417547,23.452757,...,24.847803,24.895730,24.813605,24.756353,24.814577,24.797611,24.806853,24.905993,25.033522,25.212718


In [13]:
print("전체 지역 수 (원본):", df.shape[0])
print("실제로 테스트된 지역 수:", len(all_test_data))

전체 지역 수 (원본): 1062
실제로 테스트된 지역 수: 1062


In [19]:
print(list(all_test_data.keys())[-1])  # 실제 테스트에 사용된 지역 ID 목록
print(list(all_test_data.keys()))  # 실제 테스트에 사용된 지역 ID 목록


1062
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '15

In [20]:
split_ratio = 0.7  # 앞 70%를 train, 뒤 30%를 test

all_train_data = {}
all_test_data = {}
all_train_labels = {}
all_test_labels = {}
all_train_timestamps = {}
all_test_timestamps = {}

for region_id in df.index:
    seq = df.loc[region_id].ffill().bfill()
    if len(seq) < 100:
        continue

    T = len(seq)
    s = int(T * split_ratio)

    all_train_data[str(region_id)] = seq[:s].reshape(-1, 1)
    all_train_labels[str(region_id)] = np.zeros(s)
    all_train_timestamps[str(region_id)] = np.arange(s)

    all_test_data[str(region_id)] = seq[s:].reshape(-1, 1)
    all_test_labels[str(region_id)] = np.zeros(T - s)
    all_test_timestamps[str(region_id)] = np.arange(T - s)

/tmp/ipykernel_453365/3798758525.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  seq = df.loc[region_id].fillna(method='ffill').fillna(method='bfill').astype(np.float32).values
/tmp/ipykernel_453365/3798758525.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  seq = df.loc[region_id].fillna(method='ffill').fillna(method='bfill').astype(np.float32).values
/tmp/ipykernel_453365/3798758525.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  seq = df.loc[region_id].fillna(method='ffill').fillna(method='bfill').astype(np.float32).values
/tmp/ipykernel_453365/3798758525.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  seq = df.loc

# 당신이 말한 구조, 완전히 맞습니다!

* ✔️ 상황 요약
    * 모델은 이미 FordA 데이터로 pretrain되어 있음 → ✅ OK

    * 새로운 th 시계열 데이터를 갖고 있음

    * 정답 라벨은 없음 → 비지도 방식

    * 이상치 정의: train 시점 기준으로 test 시점이 "다르게 보이는" 것 → ✅ 맞습니다!

In [21]:
from tasks.anomaly_detection import eval_anomaly_detection

res_log, eval_res = eval_anomaly_detection(
    model,  # FordA로 학습된 모델 그대로 사용
    all_train_data,
    all_train_labels,
    all_train_timestamps,
    all_test_data,
    all_test_labels,
    all_test_timestamps,
    delay=7
)

/home/cksgh8511/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
print(res_log, eval_res)

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [24]:
print(len(res_log))
print(len(res_log[0]))

1062
605


In [27]:
res_logs = np.array(res_log)
res_logs.shape

(1062, 605)

In [28]:
def get_anomaly_ranges(binary_array):
    indices = np.where(binary_array == 1)[0]
    if len(indices) == 0:
        return []

    ranges = []
    start = indices[0]

    for i in range(1, len(indices)):
        if indices[i] != indices[i-1] + 1:
            end = indices[i-1]
            ranges.append((start, end))
            start = indices[i]
    ranges.append((start, indices[-1]))  # 마지막 구간 추가

    return ranges

In [30]:
import pandas as pd

rows = []

for region_id, preds in zip(all_test_data.keys(), res_logs):
    ranges = get_anomaly_ranges(preds)
    for start, end in ranges:
        rows.append({
            "region_id": region_id,
            "anomaly_start": start,
            "anomaly_end": end,
            "length": end - start + 1
        })

# DataFrame 생성
anomaly_df = pd.DataFrame(rows)
anomaly_df

,region_id,anomaly_start,anomaly_end,length
0,1,281,281,1
1,1,561,561,1
2,2,281,281,1
3,2,561,561,1
4,3,281,281,1
...,...,...,...,...
1267,1060,561,561,1
1268,1061,281,281,1
1269,1061,561,561,1
1270,1062,281,281,1


In [31]:
anomaly_df.to_csv("anomaly_detection_results.csv", index=False)